In [36]:
from langgraph.graph import StateGraph,END,START
from typing import TypedDict
import dotenv
dotenv.load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI


In [37]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
)

In [38]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [39]:
def generate_blog_title_outline(state : BlogState) -> BlogState:
    title = state['title']
    prompt = f"Generate a detailed outline for a blog post titled '{title}'."
    outline = model.invoke(prompt)
    state['outline'] = outline
    return state


In [40]:
def create_blog_post(state : BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']
    prompt = f"Generate a blog post based on the title '{title}' and outline '{outline}'."
    content = model.invoke(prompt)
    state['content'] = content
    return state

In [41]:
# create nodes
graph = StateGraph(BlogState)

graph.add_node('generate_blog_title_outline', generate_blog_title_outline)
graph.add_node('create_blog_post', create_blog_post)

#  create edges
graph.add_edge(START, 'generate_blog_title_outline')
graph.add_edge('generate_blog_title_outline', 'create_blog_post')
graph.add_edge('create_blog_post', END)

# execute the graph
workflow = graph.compile()

In [43]:
final_state = workflow.invoke({'title': 'The Future of AI in Everyday Life'})
# print(final_state)
print(final_state['content'])

content='Of course! Here is a full blog post generated from the detailed outline provided.\n\n---\n\n## The Future of AI in Everyday Life\n\nImagine waking up not to a blaring alarm, but to gentle, simulated sunlight as your smart window lightens. Your AI home assistant, knowing your schedule and real-time traffic, has already brewed your coffee to the perfect temperature and queued up a 5-minute news summary tailored to your professional interests and personal hobbies. This isn\'t a scene from a sci-fi blockbuster; it\'s a glimpse into the near future.\n\nRight now, we have a casual relationship with AI. We ask Siri for the weather, let Netflix pick our next show, and trust Google Maps to navigate us through traffic. But these conveniences are just the "dial-up" phase of what\'s to come. We are standing at the threshold of an era where artificial intelligence will be woven so seamlessly into the fabric of our lives that we\'ll wonder how we ever lived without it.\n\nIn this article, w